In [231]:
import pandas as pd

In [232]:
df1 = pd.read_excel("C:/Users/BVM11/Downloads/2024_attendance_data.xlsx", engine="openpyxl", sheet_name='Sheet1')
df2 = pd.read_excel("C:/Users/BVM11/Downloads/2019_attendance_data.xlsx", engine="openpyxl")
df3 = pd.read_excel("C:/Users/BVM11/Downloads/2020_attendance_data.xlsx", engine="openpyxl")
df4 = pd.read_excel("C:/Users/BVM11/Downloads/2021_attendance_data.xlsx", engine="openpyxl")
df5 = pd.read_excel("C:/Users/BVM11/Downloads/2022_attendance_data.xlsx", engine="openpyxl")
df6 = pd.read_excel("C:/Users/BVM11/Downloads/2023_attendance_data.xlsx", engine="openpyxl")

In [233]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299833 entries, 0 to 299832
Data columns (total 17 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SCHOOL_YEAR                  299833 non-null  object 
 1   DISTRICT_CODE                299833 non-null  int64  
 2   DISTRICT_NAME                299833 non-null  object 
 3   LOCATION_ID                  299833 non-null  int64  
 4   SCHOOL_NAME                  299833 non-null  object 
 5   STUDENT_ID                   299833 non-null  int64  
 6   Total_Days_Unexcused_Absent  299833 non-null  float64
 7   Total_Days_Enrolled          299833 non-null  int64  
 8   Total_Days_Present           299833 non-null  float64
 9   STUDENT_GRADE_LEVEL          299833 non-null  object 
 10  CURR_GRADE_ORD               299833 non-null  object 
 11  ETHNIC_CODE                  295141 non-null  object 
 12  STUDENT_GENDER               299211 non-null  object 
 13 

In [234]:
name_map = {
    'TOTAL_DAYS_UNEXCUSED_ABSENT': 'Total_Days_Unexcused_Absent',
    'TOTAL_DAYS_ENROLLED': 'Total_Days_Enrolled',
    'TOTAL_DAYS_PRESENT': 'Total_Days_Present'
}

df1 = df1.rename(name_map, axis=1)

In [235]:
mapper = {
    'A': 'Asian',
    'B': 'Black or African American',
    'C': 'Caucasian',
    'I': 'American Indian/Alaskan Native',
    'P': 'Native Hawaiian or Other Pacific Islander',
    'H': 'Hispanic',
    'MR': 'Multi-Racial'
}

df1['ETHNIC_CODE'] = df1['ETHNIC_CODE'].replace(list(mapper.values()), list(mapper.keys()))

In [236]:
df1['ECONOMIC_CODE'] = df1['ECONOMIC_CODE'].replace([0, 1], ['N', 'F'])
df1['SPECIAL_ED_CODE'] = df1['SPECIAL_ED_CODE'].replace([0, 1], ['N', 'Y'])
df1['HISPANIC_IND'] = df1['HISPANIC_IND'].replace([0, 1], ['No', 'Yes'])
df1['STUDENT_GENDER'] = df1['STUDENT_GENDER'].replace(['M', 'F'], ['Male', 'Female'])
df1['STUDENT_GRADE_LEVEL'] = df1['STUDENT_GRADE_LEVEL'].replace(['KF', 'PK'], [0, -1]).astype('int')

C:\Users\BVM11\AppData\Local\Temp\ipykernel_24556\2517280058.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['STUDENT_GRADE_LEVEL'] = df1['STUDENT_GRADE_LEVEL'].replace(['KF', 'PK'], [0, -1]).astype('int')


In [237]:
duplicated_students =  df1[df1['STUDENT_ID'].duplicated(keep=False)].copy()

grade_pattern_ids = []
school_pattern_ids = []
grade_and_school_pattern_ids = []
ethnic_pattern_ids = []

for student, group in duplicated_students.groupby('STUDENT_ID'):
    if len(group) < 2:
        continue

    if group['STUDENT_GRADE_LEVEL'].nunique() > 1 and group['SCHOOL_NAME'].nunique() == 1:
        grade_pattern_ids.append(student)

    elif group['SCHOOL_NAME'].nunique() > 1 and group['STUDENT_GRADE_LEVEL'].nunique() == 1:
        school_pattern_ids.append(student)
    
    elif group['SCHOOL_NAME'].nunique() > 1 and group['STUDENT_GRADE_LEVEL'].nunique() > 1:
        grade_and_school_pattern_ids.append(student)
    
    else:
        ethnic_pattern_ids.append(student)


grade_pattern_df = duplicated_students[duplicated_students['STUDENT_ID'].isin(grade_pattern_ids)]
school_pattern_df = duplicated_students[duplicated_students['STUDENT_ID'].isin(school_pattern_ids)]
grade_and_school_pattern_df = duplicated_students[duplicated_students['STUDENT_ID'].isin(grade_and_school_pattern_ids)]
ethnic_pattern_df = duplicated_students[duplicated_students['STUDENT_ID'].isin(ethnic_pattern_ids)]

In [238]:
ethnic_pattern_df = ethnic_pattern_df.drop_duplicates(subset='STUDENT_ID', keep='first')
school_pattern_df = school_pattern_df.drop_duplicates(subset=['STUDENT_ID', 'SCHOOL_NAME'], keep='last').fillna('C')
grade_and_school_pattern_df = grade_and_school_pattern_df.drop_duplicates(subset='STUDENT_ID', keep='last')

In [239]:
df6['STUDENT_GRADE_LEVEL'] = df6['STUDENT_GRADE_LEVEL'].replace(['KF', 'PK'], [0, -1])
df6_unique = df6.drop_duplicates(subset=['STUDENT_ID'], keep='first')

df = (
    grade_pattern_df
    .merge(df6_unique, on="STUDENT_ID", how="left", suffixes=("", "_prev"))
)

df["expected_grade"] = df["STUDENT_GRADE_LEVEL_prev"] + 1
df["has_prev"] = df["STUDENT_GRADE_LEVEL_prev"].notna()

keepers = df.loc[
      (df["STUDENT_GRADE_LEVEL"] == df["expected_grade"])
   | (~df["has_prev"] & (df["STUDENT_GRADE_LEVEL"] == -1))
].copy()

flagged = df.drop(index=keepers.index).copy()

keepers = keepers[grade_pattern_df.columns]
flagged = flagged[grade_pattern_df.columns.to_list()]

In [240]:
flagged = flagged[~flagged['STUDENT_ID'].isin(keepers['STUDENT_ID'].values)]
idx = flagged.groupby('STUDENT_ID')['STUDENT_GRADE_LEVEL'].idxmax()
flagged = flagged.loc[idx].reset_index(drop=True)

In [241]:
dropping_ids = grade_and_school_pattern_ids + grade_pattern_ids + school_pattern_ids + ethnic_pattern_ids
df1 = df1[~df1['STUDENT_ID'].isin(dropping_ids)]
df1 = pd.concat([df1, ethnic_pattern_df, school_pattern_df, keepers, flagged, grade_and_school_pattern_df], axis=0).reset_index(drop=True)

In [242]:
gender_lookup = (
    df6
      .dropna(subset=["STUDENT_GENDER"])
      .groupby(["SCHOOL_NAME", "STUDENT_ID"], sort=False)["STUDENT_GENDER"]
      .first()
)

mask = df1["STUDENT_GENDER"].isna()
keys = list(zip(df1.loc[mask, "SCHOOL_NAME"], df1.loc[mask, "STUDENT_ID"]))
df1.loc[mask, "STUDENT_GENDER"] = [gender_lookup.get(k, pd.NA) for k in keys]
df1['STUDENT_GENDER'] = df1['STUDENT_GENDER'].fillna(df1['STUDENT_GENDER'].mode().values[0])
df1['ETHNIC_CODE'] = df1['ETHNIC_CODE'].fillna('C')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299511 entries, 0 to 299510
Data columns (total 17 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SCHOOL_YEAR                  299511 non-null  object 
 1   DISTRICT_CODE                299511 non-null  int64  
 2   DISTRICT_NAME                299511 non-null  object 
 3   LOCATION_ID                  299511 non-null  int64  
 4   SCHOOL_NAME                  299511 non-null  object 
 5   STUDENT_ID                   299511 non-null  int64  
 6   Total_Days_Unexcused_Absent  299511 non-null  float64
 7   Total_Days_Enrolled          299511 non-null  int64  
 8   Total_Days_Present           299511 non-null  float64
 9   STUDENT_GRADE_LEVEL          299511 non-null  int64  
 10  CURR_GRADE_ORD               299511 non-null  object 
 11  ETHNIC_CODE                  299511 non-null  object 
 12  STUDENT_GENDER               299511 non-null  object 
 13 

In [243]:
df1 = df1.drop(72159, axis=0)  #Dropped a record whose total days enrolled > 365

In [244]:
duplicated_past = df3[df3['STUDENT_ID'].duplicated(keep=False)].sort_values(by='STUDENT_ID')

non_duplicates = []
others = []
duplicates = []
for student, group in duplicated_past.groupby('STUDENT_ID'):
    if len(group) < 2:
        continue

    if group['SCHOOL_NAME'].nunique() == 1:
        non_duplicates.append(student)
    
    elif group['SCHOOL_NAME'].nunique() > 1:
        duplicates.append(student)
        
    else:
        others.append(student)

non_dup = duplicated_past[duplicated_past['STUDENT_ID'].isin(non_duplicates)]
dup = duplicated_past[duplicated_past['STUDENT_ID'].isin(duplicates)]
other = duplicated_past[duplicated_past['STUDENT_ID'].isin(others)]

In [245]:
df['DISTRICT_CODE'] = df['DISTRICT_CODE'].astype(str).str.zfill(3)
df['LOCATION_ID'] = df['LOCATION_ID'].astype(str).str.zfill(3)

df = pd.concat([df2, df3, df4, df5, df6, df1], axis=0).reset_index(drop=True)
df.to_csv('C:/Users/BVM11/Downloads/CompleteData.csv', index=False)